In [213]:
##############################
### Place all imports here ###
##############################
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import os
import sys
from elpv_reader import load_dataset
from PIL import Image, ImageOps

# Method 1:

# Method 2:

In [214]:
images, proba, types = load_dataset()

In [215]:
tempimages = images[:1000]
tempproba = proba[:1000]
temptypes = types[:1000]

testing_images = images[1000:]
testing_proba = proba[1000:]
testing_types = types[1000:]

In [216]:
training_images = np.zeros((3000, 300, 300))
training_proba = np.zeros(3000)
training_types = np.chararray(3000, itemsize=4)

for i in range(1000):
    training_images[i] = tempimages[i]
    training_proba[i] = tempproba[i]
    training_types[i] = temptypes[i]
    
    image = Image.fromarray(tempimages[i])
    imflip = ImageOps.flip(image)
    flipped = np.array(imflip)
    training_images[i + 1000] = flipped
    training_proba[i + 1000] = tempproba[i]
    training_types[i + 1000] = temptypes[i]
    
    image2 = Image.fromarray(tempimages[i])
    imflip2 = ImageOps.mirror(image2)
    flipped2 = np.array(imflip2)
    training_images[i + 2000] = flipped2
    training_proba[i + 2000] = tempproba[i]
    training_types[i + 2000] = temptypes[i]

In [217]:
train_dataset = tf.data.Dataset.from_tensor_slices((training_images, training_proba, training_types))
train_dataset = train_dataset.shuffle(3000).batch(32)

test_dataset = tf.data.Dataset.from_tensor_slices((testing_images, testing_proba, testing_types))
test_dataset = test_dataset.batch(32)

In [218]:
train_iterator = train_dataset.as_numpy_iterator()
train_batch = next(train_iterator)